In [1]:
import pandas as pd


In [2]:
df=pd.read_csv("./dataset/Fake news/train.csv")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
len(df)

20800

In [6]:
df=df.dropna()

In [7]:
X=df.drop('label',axis=1)
y=df["label"]

In [8]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [9]:
X.shape

(18285, 4)

In [10]:
y.shape

(18285,)

In [11]:
import tensorflow as tf

In [12]:
tf.__version__

'2.3.0'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [14]:
voc_size=5000

In [15]:
messages=X.copy()

In [16]:
messages.reset_index(inplace=True)

In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\diksh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
    
    review=re.sub('[^a-zA-Z]'," ", messages['title'][i])
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=" ".join(review)
    corpus.append(review)

In [20]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [21]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[159, 1181, 3036, 1557, 2550, 4646, 4482, 4839, 598, 404],
 [2808, 1270, 3342, 929, 3702, 4068, 4107],
 [3633, 1096, 204, 3110],
 [325, 1085, 2157, 1326, 1537, 2588],
 [211, 3702, 2199, 1533, 1027, 4182, 3702, 1182, 2467, 4737],
 [3520,
  1131,
  3917,
  4709,
  3035,
  4337,
  544,
  4242,
  103,
  2329,
  1612,
  124,
  1421,
  1377,
  4107],
 [2985, 1479, 669, 3438, 3598, 4590, 2624, 4687, 2281, 3977, 4535],
 [4455, 1669, 3080, 1584, 2247, 2613, 4337, 4928, 2281, 3977, 4535],
 [4196, 1083, 887, 2038, 2541, 4888, 2686, 2527, 4337, 3927],
 [2756, 4140, 1482, 1127, 2482, 2700, 4534, 711],
 [4418, 2680, 4990, 1613, 2001, 1360, 2949, 1483, 4089, 161, 109],
 [1326, 2216, 2550, 4888, 4337, 2247],
 [1294, 2185, 762, 4185, 4733, 4498, 4149, 2607, 12],
 [2361, 2243, 3785, 4583, 2443, 1905, 3117, 2281, 3977, 4535],
 [3212, 2452, 793, 237, 1457, 2281, 3977, 4535],
 [4190, 2707, 1095, 2659, 3397, 3726, 3387, 1198, 1104, 18],
 [4845, 2139, 1270],
 [4990, 3969, 175, 1780, 4337, 4426, 4723, 4107],

In [22]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding="pre",maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4839  598  404]
 [   0    0    0 ... 3702 4068 4107]
 [   0    0    0 ... 1096  204 3110]
 ...
 [   0    0    0 ... 2281 3977 4535]
 [   0    0    0 ... 3431 4137 4191]
 [   0    0    0 ...  920 1800 3617]]


In [23]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  159,
       1181, 3036, 1557, 2550, 4646, 4482, 4839,  598,  404])

In [24]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss="binary_crossentrophy",optimizer="adam",metrics=["accuracy"])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dense(1,activation="sigmoid"))
model1.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               112800    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [32]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)




In [33]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 27ms/step - loss: 0.3136 - accuracy: 0.8547 - val_loss: 0.1970 - val_accuracy: 0.9167
Epoch 2/10
192/192 [==============================] - 4s 21ms/step - loss: 0.1317 - accuracy: 0.9478 - val_loss: 0.1992 - val_accuracy: 0.9208
Epoch 3/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0779 - accuracy: 0.9695 - val_loss: 0.2499 - val_accuracy: 0.9196
Epoch 4/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0504 - accuracy: 0.9815 - val_loss: 0.3122 - val_accuracy: 0.9099
Epoch 5/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0325 - accuracy: 0.9901 - val_loss: 0.3863 - val_accuracy: 0.9094
Epoch 6/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0210 - accuracy: 0.9932 - val_loss: 0.3994 - val_accuracy: 0.9158
Epoch 7/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0147 - accuracy: 0.9960 - val_loss: 0.5109 - val_accuracy: 0.9112

In [34]:
y_pred1=model1.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [36]:
from sklearn.metrics import confusion_matrix

In [37]:
confusion_matrix(y_test,y_pred1)

array([[3110,  309],
       [ 227, 2389]], dtype=int64)

In [38]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.9111847555923778

In [40]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



# Dropout

In [42]:
from tensorflow.keras.layers import Dropout

In [43]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model1.add(Dense(1,activation="sigmoid"))
model1.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
print(model1.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 200)               112800    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [44]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)


In [45]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 33ms/step - loss: 0.3088 - accuracy: 0.8518 - val_loss: 0.1986 - val_accuracy: 0.9167
Epoch 2/10
192/192 [==============================] - 4s 23ms/step - loss: 0.1320 - accuracy: 0.9473 - val_loss: 0.2254 - val_accuracy: 0.9143
Epoch 3/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0856 - accuracy: 0.9677 - val_loss: 0.2281 - val_accuracy: 0.9163
Epoch 4/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0553 - accuracy: 0.9807 - val_loss: 0.2944 - val_accuracy: 0.9168
Epoch 5/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0289 - accuracy: 0.9905 - val_loss: 0.4566 - val_accuracy: 0.9133
Epoch 6/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0236 - accuracy: 0.9916 - val_loss: 0.3984 - val_accuracy: 0.9147
Epoch 7/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0149 - accuracy: 0.9961 - val_loss: 0.5692 - val_accuracy: 0.9021

In [46]:
y_pred1=model1.predict_classes(X_test)

In [47]:
confusion_matrix(y_test,y_pred1)

array([[3108,  311],
       [ 257, 2359]], dtype=int64)

In [48]:
accuracy_score(y_test,y_pred1)

0.9058823529411765

In [49]:
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      3419
           1       0.88      0.90      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.91      0.90      6035
weighted avg       0.91      0.91      0.91      6035

